# TEXT MINING for PRACTICE
- 본 자료는 텍스트 마이닝을 활용한 연구 및 강의를 위한 목적으로 제작되었습니다.
- 본 자료를 강의 목적으로 활용하고자 하시는 경우 꼭 아래 메일주소로 연락주세요.
- 본 자료에 대한 허가되지 않은 배포를 금지합니다.
- 강의, 저작권, 출판, 특허, 공동저자에 관련해서는 문의 바랍니다.
- **Contact : ADMIN(admin@teanaps.com)**

---

## WEEK 08-1. 텍스트 분류 (Text Classification)
- Python으로 텍스트 분류를 수행하는 방법에 대해 다룹니다.

---

In [1]:
# 텍스트 분석을 위한 TEANAPS 패키지를 설치합니다.
# TEANAPS는 Google Colaboratory/Linux 환경에 최적화되어 있습니다.
# Windows 환경에서 일부 기능에 제한이 있을 수 있습니다.

In [2]:
# TEANAPS (https://github.com/fingeredman/teanaps)
#!git clone https://github.com/fingeredman/teanaps.git

In [3]:
#!ls

In [4]:
# TEANAPS 설치를 진행합니다.
# 설치 전 반드시 상단 메뉴에서 [런타임 > 런타임 초기화]를 클릭한 후 진행해주세요.
#!python "teanaps/teanaps_setup.py"

### 1. 학습 데이터 준비하기

---

#### 1.1. TEANAPS 라이브러리 불러오기

---

In [1]:
from teanaps.nlp import MorphologicalAnalyzer
from teanaps.nlp import Processing
from teanaps.handler import FileHandler
from teanaps.text_analysis import TfidfCalculator

from sklearn.model_selection import train_test_split

ma = MorphologicalAnalyzer()
ma.set_tagger("mecab")
tfidf = TfidfCalculator()
processing = Processing()
fh = FileHandler()

#### 1.2. 데이터 파일 불러오기

---

In [2]:
data_path = "data/chat_intent.txt"
data = fh.load_txt(data_path)

In [3]:
print(len(data))

612


In [4]:
data[:5]

[['몇시야', 'date'],
 ['몇시지', 'date'],
 ['몇시일까', 'date'],
 ['몇시게', 'date'],
 ['시간', 'date']]

#### 1.3. 데이터 전처리하기

---

In [5]:
intent_id_to_name = {}
intent_name_to_id = {}
intent_id = -1
temp_intent_name = None

data_list = []

index = 0
for query, intent_name in data:
    index += 1
    print(index, end="\r")
    
    # 클래스 이름이 다른경우 클래스 ID(intent_id)를 업데이트
    if temp_intent_name != intent_name:
        intent_id += 1
    temp_intent_name = intent_name
    intent_id_to_name[intent_id] = intent_name
    intent_name_to_id[intent_name] = intent_id
    
    # 형태소분석 후 data_list 추가
    query_lower = query.lower()
    pos_result = ma.parse(query_lower)
    data_list.append(processing.get_plain_text(pos_result, tag=False))

In [6]:
intent_id_to_name

{0: 'date', 1: 'weather', 2: 'restraunt'}

In [7]:
intent_name_to_id

{'date': 0, 'weather': 1, 'restraunt': 2}

In [8]:
data_list[:10]

['몇 시야',
 '몇 시지',
 '몇 시 일까',
 '몇 시게',
 '시간',
 '몇 시',
 '시간 알려줘',
 '지금 몇 시야',
 '지금 몇 시지',
 '몇 시 인지 알 아']

#### 1.4. 텍스트 데이터 임베딩: TF-IDF Matrix

---

In [9]:
# TF-IDF 계산
tfidf.calculation_tfidf(data_list, 
                        tf_vectorizer_path="data/tf_vectorizer", 
                        tfidf_vectorizer_path="data/tfidf_vectorizer")
tfidf_matrix = tfidf.get_tfidf_matrix().values[:]

# 정답 label 불러오기
label_list = [intent_name_to_id[intent_name] for _, intent_name in data]

# 학습데이터/평가데이터 분리
x_train, x_test, y_train, y_test = train_test_split(tfidf_matrix, label_list, test_size=0.10, random_state=None)

### 2. 학습하기

---

#### 2.1. 학습: 랜덤 포레스트 (Random Forest)

---

In [10]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier()
random_forest.fit(x_train, y_train)         # 학습
score = random_forest.score(x_test, y_test) # 평가
print("Accuracy:", score)

# 학습모델 저장
fh.save_data("data/rf_model", random_forest)

Accuracy: 0.967741935483871


#### 2.2. 학습: 서포트 벡터 머신 (SVM)

---

In [11]:
from sklearn.svm import SVC

svm_model = SVC(probability=True)
svm_model.fit(x_train, y_train)         # 학습
score = svm_model.score(x_test, y_test) # 평가
print('Accuracy :', score)

# 학습모델 저장
fh.save_data("data/svm_model", svm_model)

Accuracy : 0.967741935483871


#### 2.3. 학습: KNN

---

In [12]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=20)
knn_model.fit(x_train, y_train)         # 학습
score = knn_model.score(x_test, y_test) # 평가
print('Accuracy :', score)

# 학습모델 저장
fh.save_data("data/knn_model", knn_model)

Accuracy : 0.9032258064516129


#### 2.4. 학습: LDA

---

In [13]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda_model = LinearDiscriminantAnalysis()
lda_model.fit(x_train, y_train)         # 학습
score = lda_model.score(x_test, y_test) # 평가
print('Accuracy :', score)

# 학습모델 저장
fh.save_data("data/lda_model", lda_model)

Accuracy : 0.9354838709677419


### 3. 학습모델 활용하기

---

In [14]:
query = "오늘 며칠이지?"
#query = "근처 맛집 추천해줘"
#query = "지금 비와?"

query_lower = query.lower()
pos_result = ma.parse(query_lower)
input_vector = tfidf.get_tfidf_vector(processing.get_plain_text(pos_result, tag=False), 
                                      tfidf_vectorizer_path="data/tfidf_vectorizer")
# 학습모델 불러오기
model = fh.load_data("data/svm_model")
# 분류수행
intent_prob_list = model.predict_proba([input_vector]).tolist()[0]

intent_list = [(intent_id_to_name[i], i, r) for i, r in enumerate(intent_prob_list)]
intent_list.sort(key=lambda elem: elem[2], reverse=True)
for intent_no, intent in enumerate(intent_list):
    print(intent_no, intent)

0 ('date', 0, 0.9980999093087556)
1 ('weather', 1, 0.001675086264531024)
2 ('restraunt', 2, 0.0002250044267133212)
